# Generate Figures for Manuscript


# Setup

In [ ]:
## Load packages
import grama as gr
import numpy as np
import pandas as pd
%matplotlib inline

## Load scikit tools
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern

## Package setup
DF = gr.Intention()

## Helper Functions

In [ ]:
## Barycentric converter
def tran_bary(df, a="I", b="Br", c="Cl", x="x", y="y"):
    r"""Convert to barycentric coordinates
    """
    df_tmp = df.copy()
    df_tmp[x] = 0.0 * df[a] + 1.0 * df[b] + 0.5 * df[c]
    df_tmp[y] = 0.0 * df[a] + 0.0 * df[b] + np.sqrt(1 - 0.5**2) * df[c]
    
    return df_tmp

## Inverse transform
def tran_thp(df, x="x", y="y", a="I", b="Br", c="Cl"):
    df_tmp = df.copy()
    xa = 0; ya = 0
    xb = 1; yb = 0
    xc = 0.5; yc = np.sqrt(1 - 0.5**2)
    d = (yb - yc) * (xa - xc) + (xc - xb) * (ya - yc)
    df_tmp[a] = ((yb - yc) * (df_tmp[x] - xc) + (xc - xb) * (df_tmp[y] - yc)) / d
    df_tmp[b] = ((yc - ya) * (df_tmp[x] - xc) + (xa - xc) * (df_tmp[y] - yc)) / d
    df_tmp[c] = 1 - df_tmp[a] - df_tmp[b]

    return df_tmp

# Pipe-enabled functions
tf_bary = gr.add_pipe(tran_bary)
tf_thp = gr.add_pipe(tran_thp)

## Metadata


In [ ]:
# Triangle boundary
df_tri_bounds = (
    gr.df_make(
        Br=[1, 0, 0, 1],
        Cl=[0, 1, 0, 0],
        I =[0, 0, 1, 0],
    )
    >> tf_bary()
)

# Labels
df_tri_labels = (
    gr.df_make(
        x=[-0.1, +1.1, +0.5],
        y=[+0.0, +0.0, +np.sqrt(1 - 0.5**2) + 0.1],
        label=["I", "Br", "Cl"]
    )
)

# Guidelines
df_tri_guides = (
    gr.df_make(
        Br_0=[0.25, 0.5, 0.75, 0.25, 0.5, 0.75, 0.75, 0.5, 0.25],
        Br_1=[0.25, 0.5, 0.75, 0.00, 0.0, 0.00, 0.00, 0.0, 0.00],
        Cl_0=[0.75, 0.5, 0.25, 0.75, 0.5, 0.25, 0.00, 0.0, 0.00],
        Cl_1=[0.00, 0.0, 0.00, 0.75, 0.5, 0.25, 0.75, 0.5, 0.25],
        I_0 =[0.00, 0.0, 0.00, 0.00, 0.0, 0.00, 0.25, 0.5, 0.75],
        I_1 =[0.75, 0.5, 0.25, 0.25, 0.5, 0.75, 0.25, 0.5, 0.75],
    )
    >> tf_bary(a="I_0", b="Br_0", c="Cl_0")
    >> tf_bary(a="I_1", b="Br_1", c="Cl_1", x="xend", y="yend")
)

## Load measurements

In [ ]:
df_data = (
    pd.read_csv("./Data/measurements.csv")
    >> gr.tf_arrange(DF["round"])
)
df_data

# Generate Figures


## Primary ternary figure


In [ ]:
p = (
    df_data
    >> tf_bary()
    
    >> gr.ggplot(gr.aes("x", "y"))
    # Guides
    + gr.geom_segment(
        data=df_tri_guides,
        mapping=gr.aes(xend="xend", yend="yend"),
        color="grey",
        size=0.5,
    )
    + gr.geom_segment(
        data=df_tri_bounds,
        mapping=gr.aes(xend="gr.lead(x)", yend="gr.lead(y)"),
        size=1.0,
    )
    + gr.geom_label(
        data=df_tri_labels,
        mapping=gr.aes(label="label")
    )
    # Data
    + gr.geom_point(gr.aes(color="single_phase"), size=6)
    # Plot tail
    + gr.theme_void()
    + gr.theme(
        legend_position="bottom",
        plot_background=gr.element_rect(fill="white")
    )
)
p.save("./images/ternary-data.png")
p